In [1]:
import pandas as pd
import numpy as np

In [53]:
from pandas.api.types import CategoricalDtype

In [163]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer   

In [10]:
data = pd.read_excel("wdi2018.xlsx")

In [11]:
data

,Time,Time Code,Country Name,Country Code,Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS],"Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]","Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]",Access to electricity (% of population) [EG.ELC.ACCS.ZS],"Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]","Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]",...,Women who believe a husband is justified in beating his wife (any of five reasons) (%) [SG.VAW.REAS.ZS],Women who believe a husband is justified in beating his wife when she argues with him (%) [SG.VAW.ARGU.ZS],Women who believe a husband is justified in beating his wife when she burns the food (%) [SG.VAW.BURN.ZS],Women who believe a husband is justified in beating his wife when she goes out without telling him (%) [SG.VAW.GOES.ZS],Women who believe a husband is justified in beating his wife when she neglects the children (%) [SG.VAW.NEGL.ZS],Women who believe a husband is justified in beating his wife when she refuses sex with him (%) [SG.VAW.REFU.ZS],Women who were first married by age 15 (% of women ages 20-24) [SP.M15.2024.FE.ZS],Women who were first married by age 18 (% of women ages 20-24) [SP.M18.2024.FE.ZS],Women's share of population ages 15+ living with HIV (%) [SH.DYN.AIDS.FE.ZS],Young people (ages 15-24) newly infected with HIV [SH.HIV.INCD.YG]
0,2018,YR2018,Afghanistan,AFG,30.9,13.85,81.6,96.616135,95.586174,99.626022,...,..,..,..,..,..,..,..,..,29,500
1,2018,YR2018,Albania,ALB,79.8,62.5,92.2,100,100,100,...,6.8,1.8,0.8,3.7,5.2,0.9,1.4,11.8,25.9,100
2,2018,YR2018,Algeria,DZA,99.6,98.7,100,99.697838,99.071304,99.933952,...,..,..,..,..,..,..,..,..,46.5,500
3,2018,YR2018,American Samoa,ASM,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,2018,YR2018,Andorra,AND,100,100,100,100,100,100,...,..,..,..,..,..,..,..,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,2018,YR2018,Sub-Saharan Africa,SSF,16.250483,5.54069,32.865363,46.267489,26.955024,73.688568,...,..,..,..,..,..,..,..,..,60.654807,..
262,2018,YR2018,Sub-Saharan Africa (excluding high income),SSA,16.243064,5.534628,32.856981,46.262729,26.948988,73.684042,...,..,..,..,..,..,..,..,..,60.654807,..
263,2018,YR2018,Sub-Saharan Africa (IDA & IBRD countries),TSS,16.250483,5.54069,32.865363,46.267489,26.955024,73.688568,...,..,..,..,..,..,..,..,..,60.654807,..
264,2018,YR2018,Upper middle income,UMC,81.734674,63.792109,91.377499,99.238818,98.74761,99.622869,...,..,..,..,..,..,..,..,..,..,..


In [15]:
data = data.replace(to_replace='...', value=np.NaN)

In [16]:
data

,Time,Time Code,Country Name,Country Code,Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS],"Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]","Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]",Access to electricity (% of population) [EG.ELC.ACCS.ZS],"Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]","Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]",...,Women who believe a husband is justified in beating his wife (any of five reasons) (%) [SG.VAW.REAS.ZS],Women who believe a husband is justified in beating his wife when she argues with him (%) [SG.VAW.ARGU.ZS],Women who believe a husband is justified in beating his wife when she burns the food (%) [SG.VAW.BURN.ZS],Women who believe a husband is justified in beating his wife when she goes out without telling him (%) [SG.VAW.GOES.ZS],Women who believe a husband is justified in beating his wife when she neglects the children (%) [SG.VAW.NEGL.ZS],Women who believe a husband is justified in beating his wife when she refuses sex with him (%) [SG.VAW.REFU.ZS],Women who were first married by age 15 (% of women ages 20-24) [SP.M15.2024.FE.ZS],Women who were first married by age 18 (% of women ages 20-24) [SP.M18.2024.FE.ZS],Women's share of population ages 15+ living with HIV (%) [SH.DYN.AIDS.FE.ZS],Young people (ages 15-24) newly infected with HIV [SH.HIV.INCD.YG]
0,2018,YR2018,Afghanistan,AFG,30.900000,13.850000,81.600000,96.616135,95.586174,99.626022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.000000,500.0
1,2018,YR2018,Albania,ALB,79.800000,62.500000,92.200000,100.000000,100.000000,100.000000,...,6.8,1.8,0.8,3.7,5.2,0.9,1.4,11.8,25.900000,100.0
2,2018,YR2018,Algeria,DZA,99.600000,98.700000,100.000000,99.697838,99.071304,99.933952,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.500000,500.0
3,2018,YR2018,American Samoa,ASM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,YR2018,Andorra,AND,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,2018,YR2018,Sub-Saharan Africa,SSF,16.250483,5.540690,32.865363,46.267489,26.955024,73.688568,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.654807,NaN
262,2018,YR2018,Sub-Saharan Africa (excluding high income),SSA,16.243064,5.534628,32.856981,46.262729,26.948988,73.684042,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.654807,NaN
263,2018,YR2018,Sub-Saharan Africa (IDA & IBRD countries),TSS,16.250483,5.540690,32.865363,46.267489,26.955024,73.688568,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.654807,NaN
264,2018,YR2018,Upper middle income,UMC,81.734674,63.792109,91.377499,99.238818,98.747610,99.622869,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
#% of missing values per column 

#Because division of sum by length is mean, you can instead df_missing.sum()/len(df) use df_missing.mean()
result = data.isna().mean()

In [18]:
print(result)

Time                                                                                                               0.000000
Time Code                                                                                                          0.000000
Country Name                                                                                                       0.000000
Country Code                                                                                                       0.000000
Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS]                              0.109023
                                                                                                                     ...   
Women who believe a husband is justified in beating his wife when she refuses sex with him (%) [SG.VAW.REFU.ZS]    0.958647
Women who were first married by age 15 (% of women ages 20-24) [SP.M15.2024.FE.ZS]                                 0.890977
Women wh

In [19]:
#remove columns with >20% NA
data_filtered80 = data.loc[:,result < .2]
#print(list(data_filtered80.columns))

print(len(list(data_filtered80.columns)))

448


In [23]:
income_group = pd.read_csv("income.csv")

In [24]:
income_group

,country code,country,income
0,AFG,Afghanistan,L
1,ALB,Albania,UM
2,DZA,Algeria,UM
3,ASM,American Samoa,UM
4,AND,Andorra,H
...,...,...,...
213,VIR,Virgin Islands (U.S.),H
214,PSE,West Bank and Gaza,LM
215,YEM,"Yemen, Rep.",L
216,ZMB,Zambia,LM


In [37]:
income_group = income_group.rename(columns={"country code": "Country Code", "country": "Country Name", "income" : "Income Group"})


In [38]:
income_group.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country Code  218 non-null    object
 1   Country Name  218 non-null    object
 2   Income Group  218 non-null    object
dtypes: object(3)
memory usage: 5.2+ KB


In [48]:
income_group = income_group.drop(['Country Name'], axis=1)

In [39]:
data_filtered80.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 448 columns):
 #    Column                                                                                                                                           Dtype  
---   ------                                                                                                                                           -----  
 0    Time                                                                                                                                             int64  
 1    Time Code                                                                                                                                        object 
 2    Country Name                                                                                                                                     object 
 3    Country Code                                                                                 

In [40]:
full_data = data_filtered80

In [69]:
full_data_plus_income_group = pd.merge(full_data, income_group, on = "Country Code")

In [70]:
full_data_plus_income_group

,Time,Time Code,Country Name,Country Code,Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS],"Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]","Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]",Access to electricity (% of population) [EG.ELC.ACCS.ZS],"Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]","Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]",...,Urban population (% of total population) [SP.URB.TOTL.IN.ZS],Urban population growth (annual %) [SP.URB.GROW],"Vulnerable employment, female (% of female employment) (modeled ILO estimate) [SL.EMP.VULN.FE.ZS]","Vulnerable employment, male (% of male employment) (modeled ILO estimate) [SL.EMP.VULN.MA.ZS]","Vulnerable employment, total (% of total employment) (modeled ILO estimate) [SL.EMP.VULN.ZS]","Wage and salaried workers, female (% of female employment) (modeled ILO estimate) [SL.EMP.WORK.FE.ZS]","Wage and salaried workers, male (% of male employment) (modeled ILO estimate) [SL.EMP.WORK.MA.ZS]","Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]","Water productivity, total (constant 2015 US$ GDP per cubic meter of total freshwater withdrawal) [ER.GDP.FWTL.M3.KD]",Income Group
0,2018,YR2018,Afghanistan,AFG,30.9,13.85,81.6,96.616135,95.586174,99.626022,...,25.495,3.850833,89.920004,76.949996,79.619999,7.870000,20.200001,17.660000,1.047290,L
1,2018,YR2018,Albania,ALB,79.8,62.50,92.2,100.000000,100.000000,100.000000,...,60.319,1.317162,51.700001,52.540001,52.190001,47.020000,42.730000,44.520000,13.259350,UM
2,2018,YR2018,Algeria,DZA,99.6,98.70,100.0,99.697838,99.071304,99.933952,...,72.629,2.700947,23.400001,28.090001,27.349999,74.580002,66.790001,68.019997,17.914671,UM
3,2018,YR2018,American Samoa,ASM,NaN,NaN,NaN,NaN,NaN,NaN,...,87.153,-2.142604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UM
4,2018,YR2018,Andorra,AND,100.0,100.00,100.0,100.000000,100.000000,100.000000,...,88.062,1.480831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,2018,YR2018,Virgin Islands (U.S.),VIR,NaN,NaN,NaN,100.000000,100.000000,100.000000,...,95.721,-0.138546,3.220000,3.220000,3.220000,93.419998,93.410004,93.419998,NaN,H
213,2018,YR2018,West Bank and Gaza,PSE,NaN,NaN,NaN,100.000000,100.000000,100.000000,...,76.164,2.888119,22.110001,23.389999,23.180001,75.470001,68.680000,69.739998,52.106106,LM
214,2018,YR2018,"Yemen, Rep.",YEM,61.5,42.90,93.7,62.000000,NaN,NaN,...,36.642,4.209541,60.969999,46.660002,47.630000,38.029999,45.830002,45.299999,10.319535,L
215,2018,YR2018,Zambia,ZMB,11.9,2.10,24.6,40.317890,12.466436,76.461876,...,43.521,4.322060,84.240002,66.379998,74.879999,15.540000,33.340000,24.860001,15.106607,LM


In [71]:
full_data_plus_income_group['Income Group'].unique()

array(['L', 'UM', 'H', 'LM'], dtype=object)

In [72]:
category_values = ['H', 'UM', 'LM', 'L']

category_order = CategoricalDtype(category_values, ordered=True)

full_data_plus_income_group.loc[:,'Income Group'] = full_data_plus_income_group.loc[:,'Income Group'].astype(category_order)

In [73]:
full_data_plus_income_group['Income Group'].unique()

['L', 'UM', 'H', 'LM']
Categories (4, object): ['H' < 'UM' < 'LM' < 'L']

In [74]:
full_data_plus_income_group

,Time,Time Code,Country Name,Country Code,Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS],"Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]","Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]",Access to electricity (% of population) [EG.ELC.ACCS.ZS],"Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]","Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]",...,Urban population (% of total population) [SP.URB.TOTL.IN.ZS],Urban population growth (annual %) [SP.URB.GROW],"Vulnerable employment, female (% of female employment) (modeled ILO estimate) [SL.EMP.VULN.FE.ZS]","Vulnerable employment, male (% of male employment) (modeled ILO estimate) [SL.EMP.VULN.MA.ZS]","Vulnerable employment, total (% of total employment) (modeled ILO estimate) [SL.EMP.VULN.ZS]","Wage and salaried workers, female (% of female employment) (modeled ILO estimate) [SL.EMP.WORK.FE.ZS]","Wage and salaried workers, male (% of male employment) (modeled ILO estimate) [SL.EMP.WORK.MA.ZS]","Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]","Water productivity, total (constant 2015 US$ GDP per cubic meter of total freshwater withdrawal) [ER.GDP.FWTL.M3.KD]",Income Group
0,2018,YR2018,Afghanistan,AFG,30.9,13.85,81.6,96.616135,95.586174,99.626022,...,25.495,3.850833,89.920004,76.949996,79.619999,7.870000,20.200001,17.660000,1.047290,L
1,2018,YR2018,Albania,ALB,79.8,62.50,92.2,100.000000,100.000000,100.000000,...,60.319,1.317162,51.700001,52.540001,52.190001,47.020000,42.730000,44.520000,13.259350,UM
2,2018,YR2018,Algeria,DZA,99.6,98.70,100.0,99.697838,99.071304,99.933952,...,72.629,2.700947,23.400001,28.090001,27.349999,74.580002,66.790001,68.019997,17.914671,UM
3,2018,YR2018,American Samoa,ASM,NaN,NaN,NaN,NaN,NaN,NaN,...,87.153,-2.142604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UM
4,2018,YR2018,Andorra,AND,100.0,100.00,100.0,100.000000,100.000000,100.000000,...,88.062,1.480831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,2018,YR2018,Virgin Islands (U.S.),VIR,NaN,NaN,NaN,100.000000,100.000000,100.000000,...,95.721,-0.138546,3.220000,3.220000,3.220000,93.419998,93.410004,93.419998,NaN,H
213,2018,YR2018,West Bank and Gaza,PSE,NaN,NaN,NaN,100.000000,100.000000,100.000000,...,76.164,2.888119,22.110001,23.389999,23.180001,75.470001,68.680000,69.739998,52.106106,LM
214,2018,YR2018,"Yemen, Rep.",YEM,61.5,42.90,93.7,62.000000,NaN,NaN,...,36.642,4.209541,60.969999,46.660002,47.630000,38.029999,45.830002,45.299999,10.319535,L
215,2018,YR2018,Zambia,ZMB,11.9,2.10,24.6,40.317890,12.466436,76.461876,...,43.521,4.322060,84.240002,66.379998,74.879999,15.540000,33.340000,24.860001,15.106607,LM


In [75]:
full_data_plus_income_group = full_data_plus_income_group.drop(['Time Code'], axis=1)

In [76]:
full_data_plus_income_group

,Time,Country Name,Country Code,Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS],"Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]","Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]",Access to electricity (% of population) [EG.ELC.ACCS.ZS],"Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]","Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]",Adjusted net national income (current US$) [NY.ADJ.NNTY.CD],...,Urban population (% of total population) [SP.URB.TOTL.IN.ZS],Urban population growth (annual %) [SP.URB.GROW],"Vulnerable employment, female (% of female employment) (modeled ILO estimate) [SL.EMP.VULN.FE.ZS]","Vulnerable employment, male (% of male employment) (modeled ILO estimate) [SL.EMP.VULN.MA.ZS]","Vulnerable employment, total (% of total employment) (modeled ILO estimate) [SL.EMP.VULN.ZS]","Wage and salaried workers, female (% of female employment) (modeled ILO estimate) [SL.EMP.WORK.FE.ZS]","Wage and salaried workers, male (% of male employment) (modeled ILO estimate) [SL.EMP.WORK.MA.ZS]","Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]","Water productivity, total (constant 2015 US$ GDP per cubic meter of total freshwater withdrawal) [ER.GDP.FWTL.M3.KD]",Income Group
0,2018,Afghanistan,AFG,30.9,13.85,81.6,96.616135,95.586174,99.626022,1.681519e+10,...,25.495,3.850833,89.920004,76.949996,79.619999,7.870000,20.200001,17.660000,1.047290,L
1,2018,Albania,ALB,79.8,62.50,92.2,100.000000,100.000000,100.000000,1.237962e+10,...,60.319,1.317162,51.700001,52.540001,52.190001,47.020000,42.730000,44.520000,13.259350,UM
2,2018,Algeria,DZA,99.6,98.70,100.0,99.697838,99.071304,99.933952,1.368789e+11,...,72.629,2.700947,23.400001,28.090001,27.349999,74.580002,66.790001,68.019997,17.914671,UM
3,2018,American Samoa,ASM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,87.153,-2.142604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UM
4,2018,Andorra,AND,100.0,100.00,100.0,100.000000,100.000000,100.000000,NaN,...,88.062,1.480831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,2018,Virgin Islands (U.S.),VIR,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,...,95.721,-0.138546,3.220000,3.220000,3.220000,93.419998,93.410004,93.419998,NaN,H
213,2018,West Bank and Gaza,PSE,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,...,76.164,2.888119,22.110001,23.389999,23.180001,75.470001,68.680000,69.739998,52.106106,LM
214,2018,"Yemen, Rep.",YEM,61.5,42.90,93.7,62.000000,NaN,NaN,2.051920e+10,...,36.642,4.209541,60.969999,46.660002,47.630000,38.029999,45.830002,45.299999,10.319535,L
215,2018,Zambia,ZMB,11.9,2.10,24.6,40.317890,12.466436,76.461876,2.043716e+10,...,43.521,4.322060,84.240002,66.379998,74.879999,15.540000,33.340000,24.860001,15.106607,LM


In [94]:
column_list = list[full_data_plus_income_group.columns]

In [95]:
column_list

list[Index(['Time', 'Country Name', 'Country Code',
       'Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS]',
       'Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]',
       'Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]',
       'Access to electricity (% of population) [EG.ELC.ACCS.ZS]',
       'Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]',
       'Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]',
       'Adjusted net national income (current US$) [NY.ADJ.NNTY.CD]',
       ...
       'Urban population (% of total population) [SP.URB.TOTL.IN.ZS]',
       'Urban population growth (annual %) [SP.URB.GROW]',
       'Vulnerable employment, female (% of female employment) (modeled ILO estimate) [SL.EMP.VULN.FE.ZS]',
       'Vulnerable employment, male (% of male employment) (modeled ILO esti

In [100]:
'Energy intensity level of primary energy (MJ/$2017 PPP GDP)' in full_data_plus_income_group.columns

False

In [101]:
'CO2 intensity (kg per kg of oil equivalent energy use)' in full_data_plus_income_group.columns

False

In [82]:
retained_var_pol = [
    'Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS]',
    'Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]',
    'Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]',
    'Access to electricity (% of population) [EG.ELC.ACCS.ZS]',
    'Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]',
    'Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]',
    'Agricultural methane emissions (thousand metric tons of CO2 equivalent) [EN.ATM.METH.AG.KT.CE]',
    'Agricultural nitrous oxide emissions (thousand metric tons of CO2 equivalent) [EN.ATM.NOXE.AG.KT.CE]',
    'Annual freshwater withdrawals, total (% of internal resources) [ER.H2O.FWTL.ZS]',
    'CO2 emissions (kg per 2015 US$ of GDP) [EN.ATM.CO2E.KD.GD]',
    'CO2 emissions (kg per 2017 PPP $ of GDP) [EN.ATM.CO2E.PP.GD.KD]',
    'CO2 emissions (kg per PPP $ of GDP) [EN.ATM.CO2E.PP.GD]',
    'CO2 emissions (kt) [EN.ATM.CO2E.KT]',
    'CO2 emissions (metric tons per capita) [EN.ATM.CO2E.PC]',
    'Methane emissions (kt of CO2 equivalent) [EN.ATM.METH.KT.CE]',
    'Nitrous oxide emissions (thousand metric tons of CO2 equivalent) [EN.ATM.NOXE.KT.CE]',
    'Renewable energy consumption (% of total final energy consumption) [EG.FEC.RNEW.ZS]',
    'Renewable internal freshwater resources per capita (cubic meters) [ER.H2O.INTR.PC]',
    'Renewable internal freshwater resources, total (billion cubic meters) [ER.H2O.INTR.K3]',
    'Total greenhouse gas emissions (kt of CO2 equivalent) [EN.ATM.GHGT.KT.CE]',    
]

retained_var_ec = [
    'Adjusted net national income (current US$) [NY.ADJ.NNTY.CD]',
    'Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD]',
    'Adjusted savings: carbon dioxide damage (% of GNI) [NY.ADJ.DCO2.GN.ZS]',
    'Adjusted savings: consumption of fixed capital (% of GNI) [NY.ADJ.DKAP.GN.ZS]',
    'Adjusted savings: education expenditure (% of GNI) [NY.ADJ.AEDU.GN.ZS]',
    'Adjusted savings: energy depletion (% of GNI) [NY.ADJ.DNGY.GN.ZS]',
    'Adjusted savings: mineral depletion (% of GNI) [NY.ADJ.DMIN.GN.ZS]',
    'Adjusted savings: mineral depletion (current US$) [NY.ADJ.DMIN.CD]',
    'Adjusted savings: natural resources depletion (% of GNI) [NY.ADJ.DRES.GN.ZS]',
    'Adjusted savings: net forest depletion (% of GNI) [NY.ADJ.DFOR.GN.ZS]',
    'Adjusted savings: particulate emission damage (% of GNI) [NY.ADJ.DPEM.GN.ZS]',
    'Age dependency ratio (% of working-age population) [SP.POP.DPND]',
    'Age dependency ratio, old (% of working-age population) [SP.POP.DPND.OL]',
    'Age dependency ratio, young (% of working-age population) [SP.POP.DPND.YG]',
    'Contributing family workers, total (% of total employment) (modeled ILO estimate) [SL.FAM.WORK.ZS]',
    'Current health expenditure per capita, PPP (current international $) [SH.XPD.CHEX.PP.CD]',
    'Employers, total (% of total employment) (modeled ILO estimate) [SL.EMP.MPYR.ZS]',
    'Exports of goods and services (% of GDP) [NE.EXP.GNFS.ZS]',
    'Exports of goods and services (current US$) [NE.EXP.GNFS.CD]',
    'Final consumption expenditure (current US$) [NE.CON.TOTL.CD]',
    'GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]',
    'GDP (current US$) [NY.GDP.MKTP.CD]',
    'GDP per capita (current US$) [NY.GDP.PCAP.CD]',
    'GDP per person employed (constant 2017 PPP $) [SL.GDP.PCAP.EM.KD]', 
    'GDP, PPP (constant 2017 international $) [NY.GDP.MKTP.PP.KD]',
    'General government final consumption expenditure (% of GDP) [NE.CON.GOVT.ZS]',
    'General government final consumption expenditure (current US$) [NE.CON.GOVT.CD]',
    'GNI (current US$) [NY.GNP.MKTP.CD]',
    'GNI, PPP (current international $) [NY.GNP.MKTP.PP.CD]',
    'Imports of goods and services (BoP, current US$) [BM.GSR.GNFS.CD]',
    'Imports of goods and services (current US$) [NE.IMP.GNFS.CD]',
    'Industry (including construction), value added (% of GDP) [NV.IND.TOTL.ZS]',
    'Industry (including construction), value added (annual % growth) [NV.IND.TOTL.KD.ZG]',
    'Industry (including construction), value added (current US$) [NV.IND.TOTL.CD]',
    'Inflation, consumer prices (annual %) [FP.CPI.TOTL.ZG]',
    'Inflation, GDP deflator (annual %) [NY.GDP.DEFL.KD.ZG]',
    'Manufacturing, value added (% of GDP) [NV.IND.MANF.ZS]',
    'Manufacturing, value added (annual % growth) [NV.IND.MANF.KD.ZG]',
    'Manufacturing, value added (current US$) [NV.IND.MANF.CD]',
    'Population, total [SP.POP.TOTL]',
    'Total tax and contribution rate (% of profit) [IC.TAX.TOTL.CP.ZS]',
    'Unemployment, total (% of total labor force) (modeled ILO estimate) [SL.UEM.TOTL.ZS]',
    'Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]',
]

In [83]:
to_retain_vars = ['Country Code', 'Country Name'] + retained_var_pol + retained_var_ec + ['Income Group']

In [117]:
full_data_plus_income_group[to_retain_vars]

,Country Code,Country Name,Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS],"Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]","Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]",Access to electricity (% of population) [EG.ELC.ACCS.ZS],"Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]","Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]",Agricultural methane emissions (thousand metric tons of CO2 equivalent) [EN.ATM.METH.AG.KT.CE],Agricultural nitrous oxide emissions (thousand metric tons of CO2 equivalent) [EN.ATM.NOXE.AG.KT.CE],...,"Inflation, consumer prices (annual %) [FP.CPI.TOTL.ZG]","Inflation, GDP deflator (annual %) [NY.GDP.DEFL.KD.ZG]","Manufacturing, value added (% of GDP) [NV.IND.MANF.ZS]","Manufacturing, value added (annual % growth) [NV.IND.MANF.KD.ZG]","Manufacturing, value added (current US$) [NV.IND.MANF.CD]","Population, total [SP.POP.TOTL]",Total tax and contribution rate (% of profit) [IC.TAX.TOTL.CP.ZS],"Unemployment, total (% of total labor force) (modeled ILO estimate) [SL.UEM.TOTL.ZS]","Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]",Income Group
0,AFG,Afghanistan,30.9,13.85,81.6,96.616135,95.586174,99.626022,10540.0,4310.0,...,0.626149,2.071349,6.160177,22.921060,1.134634e+09,36686784.0,71.4,11.110,17.660000,L
1,ALB,Albania,79.8,62.50,92.2,100.000000,100.000000,100.000000,2010.0,960.0,...,2.028060,1.473058,6.133262,6.259229,9.295837e+08,2866376.0,37.3,12.300,44.520000,UM
2,DZA,Algeria,99.6,98.70,100.0,99.697838,99.071304,99.933952,6820.0,5530.0,...,4.269990,6.757341,26.549953,-4.603222,4.643876e+10,41927007.0,66.1,10.416,68.019997,UM
3,ASM,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.695361,13.928013,16.867470,8.900000e+07,48424.0,NaN,NaN,NaN,UM
4,AND,Andorra,100.0,100.00,100.0,100.000000,100.000000,100.000000,NaN,NaN,...,NaN,1.012478,3.610284,-2.170767,1.161904e+08,75013.0,NaN,NaN,NaN,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,VIR,Virgin Islands (U.S.),NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,NaN,...,NaN,1.441528,NaN,NaN,NaN,107001.0,NaN,12.245,93.419998,H
213,PSE,West Bank and Gaza,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,NaN,...,-0.195108,-0.301993,11.547252,0.341530,1.879500e+09,4569087.0,15.3,26.260,69.739998,LM
214,YEM,"Yemen, Rep.",61.5,42.90,93.7,62.000000,NaN,NaN,4360.0,2980.0,...,NaN,14.853665,NaN,NaN,NaN,30790513.0,26.6,13.076,45.299999,L
215,ZMB,Zambia,11.9,2.10,24.6,40.317890,12.466436,76.461876,10330.0,13510.0,...,7.494572,7.411571,6.847766,4.071350,1.801756e+09,17835893.0,15.6,5.030,24.860001,LM


In [118]:
data = full_data_plus_income_group[to_retain_vars]

In [119]:
data.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 216
Data columns (total 66 columns):
 #   Column                                                                                                 Non-Null Count  Dtype   
---  ------                                                                                                 --------------  -----   
 0   Country Code                                                                                           217 non-null    object  
 1   Country Name                                                                                           217 non-null    object  
 2   Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS]                  189 non-null    float64 
 3   Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]  189 non-null    float64 
 4   Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.AC

In [102]:
energy_intensity = pd.read_excel("energy_intensity.xlsx")

In [104]:
carbon_intensity = pd.read_csv("carbon-intensity-electricity.csv")

In [105]:
energy_intensity

,Time,Time Code,Country Name,Country Code,Energy intensity level of primary energy (MJ/$2017 PPP GDP) [EG.EGY.PRIM.PP.KD]
0,2018,YR2018,Afghanistan,AFG,2.44
1,2018,YR2018,Albania,ALB,2.57
2,2018,YR2018,Algeria,DZA,5.19
3,2018,YR2018,American Samoa,ASM,..
4,2018,YR2018,Andorra,AND,..
...,...,...,...,...,...
266,NaN,NaN,NaN,NaN,NaN
267,NaN,NaN,NaN,NaN,NaN
268,NaN,NaN,NaN,NaN,NaN
269,Data from database: World Development Indicators,NaN,NaN,NaN,NaN


In [107]:
energy_intensity.isna().mean()

Time                                                                               0.01107
Time Code                                                                          0.01845
Country Name                                                                       0.01845
Country Code                                                                       0.01845
Energy intensity level of primary energy (MJ/$2017 PPP GDP) [EG.EGY.PRIM.PP.KD]    0.01845
dtype: float64

In [122]:
energy_intensity = energy_intensity.drop(['Country Name', 'Time', 'Time Code'], axis=1)

In [123]:
energy_intensity_added = pd.merge(data, energy_intensity, on = "Country Code")

In [124]:
energy_intensity_added

,Country Code,Country Name,Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS],"Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]","Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]",Access to electricity (% of population) [EG.ELC.ACCS.ZS],"Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]","Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]",Agricultural methane emissions (thousand metric tons of CO2 equivalent) [EN.ATM.METH.AG.KT.CE],Agricultural nitrous oxide emissions (thousand metric tons of CO2 equivalent) [EN.ATM.NOXE.AG.KT.CE],...,"Inflation, GDP deflator (annual %) [NY.GDP.DEFL.KD.ZG]","Manufacturing, value added (% of GDP) [NV.IND.MANF.ZS]","Manufacturing, value added (annual % growth) [NV.IND.MANF.KD.ZG]","Manufacturing, value added (current US$) [NV.IND.MANF.CD]","Population, total [SP.POP.TOTL]",Total tax and contribution rate (% of profit) [IC.TAX.TOTL.CP.ZS],"Unemployment, total (% of total labor force) (modeled ILO estimate) [SL.UEM.TOTL.ZS]","Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]",Income Group,Energy intensity level of primary energy (MJ/$2017 PPP GDP) [EG.EGY.PRIM.PP.KD]
0,AFG,Afghanistan,30.9,13.85,81.6,96.616135,95.586174,99.626022,10540.0,4310.0,...,2.071349,6.160177,22.921060,1.134634e+09,36686784.0,71.4,11.110,17.660000,L,2.44
1,ALB,Albania,79.8,62.50,92.2,100.000000,100.000000,100.000000,2010.0,960.0,...,1.473058,6.133262,6.259229,9.295837e+08,2866376.0,37.3,12.300,44.520000,UM,2.57
2,DZA,Algeria,99.6,98.70,100.0,99.697838,99.071304,99.933952,6820.0,5530.0,...,6.757341,26.549953,-4.603222,4.643876e+10,41927007.0,66.1,10.416,68.019997,UM,5.19
3,ASM,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.695361,13.928013,16.867470,8.900000e+07,48424.0,NaN,NaN,NaN,UM,..
4,AND,Andorra,100.0,100.00,100.0,100.000000,100.000000,100.000000,NaN,NaN,...,1.012478,3.610284,-2.170767,1.161904e+08,75013.0,NaN,NaN,NaN,H,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,VIR,Virgin Islands (U.S.),NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,NaN,...,1.441528,NaN,NaN,NaN,107001.0,NaN,12.245,93.419998,H,..
213,PSE,West Bank and Gaza,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,NaN,...,-0.301993,11.547252,0.341530,1.879500e+09,4569087.0,15.3,26.260,69.739998,LM,2.55
214,YEM,"Yemen, Rep.",61.5,42.90,93.7,62.000000,NaN,NaN,4360.0,2980.0,...,14.853665,NaN,NaN,NaN,30790513.0,26.6,13.076,45.299999,L,..
215,ZMB,Zambia,11.9,2.10,24.6,40.317890,12.466436,76.461876,10330.0,13510.0,...,7.411571,6.847766,4.071350,1.801756e+09,17835893.0,15.6,5.030,24.860001,LM,7.35


In [125]:
carbon_intensity

,Entity,Code,Year,Carbon intensity of electricity (gCO2/kWh)
0,Afghanistan,AFG,2000,255.319140
1,Afghanistan,AFG,2001,118.644066
2,Afghanistan,AFG,2002,144.927540
3,Afghanistan,AFG,2003,255.319140
4,Afghanistan,AFG,2004,269.662900
...,...,...,...,...
5074,Zimbabwe,ZWE,2017,363.265320
5075,Zimbabwe,ZWE,2018,355.821560
5076,Zimbabwe,ZWE,2019,387.135930
5077,Zimbabwe,ZWE,2020,389.473700


In [126]:
carbon_intensity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5079 entries, 0 to 5078
Data columns (total 4 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Entity                                      5079 non-null   object 
 1   Code                                        4615 non-null   object 
 2   Year                                        5079 non-null   int64  
 3   Carbon intensity of electricity (gCO2/kWh)  5079 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 158.8+ KB


In [127]:
carbon_intensity_2018 = carbon_intensity[carbon_intensity['Year'] == 2018]

In [128]:
carbon_intensity_2018

,Entity,Code,Year,Carbon intensity of electricity (gCO2/kWh)
18,Afghanistan,AFG,2018,136.752140
40,Africa,NaN,2018,505.281620
62,Africa (Ember),NaN,2018,505.416530
84,Albania,ALB,2018,24.561401
105,Algeria,DZA,2018,488.975200
...,...,...,...,...
4977,Vietnam,VNM,2018,432.812300
5009,World,OWID_WRL,2018,459.112820
5031,Yemen,YEM,2018,542.424260
5053,Zambia,ZMB,2018,130.192200


In [129]:
carbon_intensity_2018 = carbon_intensity_2018.drop(['Entity', 'Year'], axis=1)

In [130]:
carbon_intensity_2018

,Code,Carbon intensity of electricity (gCO2/kWh)
18,AFG,136.752140
40,NaN,505.281620
62,NaN,505.416530
84,ALB,24.561401
105,DZA,488.975200
...,...,...
4977,VNM,432.812300
5009,OWID_WRL,459.112820
5031,YEM,542.424260
5053,ZMB,130.192200


In [136]:
carbon_intensity_2018 = carbon_intensity_2018.rename(columns={"Code": "Country Code"})


In [137]:
carbon_intensity_2018

,Country Code,Carbon intensity of electricity (gCO2/kWh)
18,AFG,136.752140
40,NaN,505.281620
62,NaN,505.416530
84,ALB,24.561401
105,DZA,488.975200
...,...,...
4977,VNM,432.812300
5009,OWID_WRL,459.112820
5031,YEM,542.424260
5053,ZMB,130.192200


In [138]:
carbon_intensity_added = pd.merge(energy_intensity_added, carbon_intensity_2018, on = "Country Code")

In [139]:
carbon_intensity_added

,Country Code,Country Name,Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS],"Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]","Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]",Access to electricity (% of population) [EG.ELC.ACCS.ZS],"Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]","Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]",Agricultural methane emissions (thousand metric tons of CO2 equivalent) [EN.ATM.METH.AG.KT.CE],Agricultural nitrous oxide emissions (thousand metric tons of CO2 equivalent) [EN.ATM.NOXE.AG.KT.CE],...,"Manufacturing, value added (% of GDP) [NV.IND.MANF.ZS]","Manufacturing, value added (annual % growth) [NV.IND.MANF.KD.ZG]","Manufacturing, value added (current US$) [NV.IND.MANF.CD]","Population, total [SP.POP.TOTL]",Total tax and contribution rate (% of profit) [IC.TAX.TOTL.CP.ZS],"Unemployment, total (% of total labor force) (modeled ILO estimate) [SL.UEM.TOTL.ZS]","Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]",Income Group,Energy intensity level of primary energy (MJ/$2017 PPP GDP) [EG.EGY.PRIM.PP.KD],Carbon intensity of electricity (gCO2/kWh)
0,AFG,Afghanistan,30.9,13.85,81.6,96.616135,95.586174,99.626022,10540.0,4310.0,...,6.160177,22.921060,1.134634e+09,36686784.0,71.4,11.110,17.660000,L,2.44,136.752140
1,ALB,Albania,79.8,62.50,92.2,100.000000,100.000000,100.000000,2010.0,960.0,...,6.133262,6.259229,9.295837e+08,2866376.0,37.3,12.300,44.520000,UM,2.57,24.561401
2,DZA,Algeria,99.6,98.70,100.0,99.697838,99.071304,99.933952,6820.0,5530.0,...,26.549953,-4.603222,4.643876e+10,41927007.0,66.1,10.416,68.019997,UM,5.19,488.975200
3,ASM,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.928013,16.867470,8.900000e+07,48424.0,NaN,NaN,NaN,UM,..,687.500000
4,AGO,Angola,49.1,8.10,77.7,45.290001,7.333445,65.269997,16220.0,16140.0,...,6.059469,0.571521,4.713840e+09,31273533.0,49.1,9.522,21.620001,LM,2.95,137.850460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,VIR,Virgin Islands (U.S.),NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,NaN,...,NaN,NaN,NaN,107001.0,NaN,12.245,93.419998,H,..,688.524540
196,PSE,West Bank and Gaza,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,NaN,...,11.547252,0.341530,1.879500e+09,4569087.0,15.3,26.260,69.739998,LM,2.55,524.999940
197,YEM,"Yemen, Rep.",61.5,42.90,93.7,62.000000,NaN,NaN,4360.0,2980.0,...,NaN,NaN,NaN,30790513.0,26.6,13.076,45.299999,L,..,542.424260
198,ZMB,Zambia,11.9,2.10,24.6,40.317890,12.466436,76.461876,10330.0,13510.0,...,6.847766,4.071350,1.801756e+09,17835893.0,15.6,5.030,24.860001,LM,7.35,130.192200


In [143]:
for e in carbon_intensity_added['Country Name']:
    print(e)

Afghanistan
Albania
Algeria
American Samoa
Angola
Antigua and Barbuda
Argentina
Armenia
Aruba
Australia
Austria
Azerbaijan
Bahamas, The
Bahrain
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bosnia and Herzegovina
Botswana
Brazil
British Virgin Islands
Brunei Darussalam
Bulgaria
Burkina Faso
Burundi
Cabo Verde
Cambodia
Cameroon
Canada
Cayman Islands
Central African Republic
Chad
Chile
China
Colombia
Comoros
Congo, Dem. Rep.
Congo, Rep.
Costa Rica
Cote d'Ivoire
Croatia
Cuba
Cyprus
Czechia
Denmark
Djibouti
Dominica
Dominican Republic
Ecuador
Egypt, Arab Rep.
El Salvador
Equatorial Guinea
Eritrea
Estonia
Eswatini
Ethiopia
Faroe Islands
Fiji
Finland
France
French Polynesia
Gabon
Gambia, The
Georgia
Germany
Ghana
Greece
Greenland
Grenada
Guam
Guatemala
Guinea
Guinea-Bissau
Guyana
Haiti
Honduras
Hong Kong SAR, China
Hungary
Iceland
India
Indonesia
Iran, Islamic Rep.
Iraq
Ireland
Israel
Italy
Jamaica
Japan
Jordan
Kazakhstan
Kenya
Kiribati
Korea, Dem. People's Rep.
Korea, Rep.

In [150]:
list_col = carbon_intensity_added.columns

In [153]:
print(list_col)

Index(['Country Code', 'Country Name',
       'Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS]',
       'Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]',
       'Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]',
       'Access to electricity (% of population) [EG.ELC.ACCS.ZS]',
       'Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]',
       'Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]',
       'Agricultural methane emissions (thousand metric tons of CO2 equivalent) [EN.ATM.METH.AG.KT.CE]',
       'Agricultural nitrous oxide emissions (thousand metric tons of CO2 equivalent) [EN.ATM.NOXE.AG.KT.CE]',
       'Annual freshwater withdrawals, total (% of internal resources) [ER.H2O.FWTL.ZS]',
       'CO2 emissions (kg per 2015 US$ of GDP) [EN.ATM.CO2E.KD.GD]',
       'CO2 emissions (kg per 201

In [152]:
print(len(list_col))

68


In [154]:
carbon_intensity_added

,Country Code,Country Name,Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS],"Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]","Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]",Access to electricity (% of population) [EG.ELC.ACCS.ZS],"Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]","Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]",Agricultural methane emissions (thousand metric tons of CO2 equivalent) [EN.ATM.METH.AG.KT.CE],Agricultural nitrous oxide emissions (thousand metric tons of CO2 equivalent) [EN.ATM.NOXE.AG.KT.CE],...,"Manufacturing, value added (% of GDP) [NV.IND.MANF.ZS]","Manufacturing, value added (annual % growth) [NV.IND.MANF.KD.ZG]","Manufacturing, value added (current US$) [NV.IND.MANF.CD]","Population, total [SP.POP.TOTL]",Total tax and contribution rate (% of profit) [IC.TAX.TOTL.CP.ZS],"Unemployment, total (% of total labor force) (modeled ILO estimate) [SL.UEM.TOTL.ZS]","Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]",Income Group,Energy intensity level of primary energy (MJ/$2017 PPP GDP) [EG.EGY.PRIM.PP.KD],Carbon intensity of electricity (gCO2/kWh)
0,AFG,Afghanistan,30.9,13.85,81.6,96.616135,95.586174,99.626022,10540.0,4310.0,...,6.160177,22.921060,1.134634e+09,36686784.0,71.4,11.110,17.660000,L,2.44,136.752140
1,ALB,Albania,79.8,62.50,92.2,100.000000,100.000000,100.000000,2010.0,960.0,...,6.133262,6.259229,9.295837e+08,2866376.0,37.3,12.300,44.520000,UM,2.57,24.561401
2,DZA,Algeria,99.6,98.70,100.0,99.697838,99.071304,99.933952,6820.0,5530.0,...,26.549953,-4.603222,4.643876e+10,41927007.0,66.1,10.416,68.019997,UM,5.19,488.975200
3,ASM,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13.928013,16.867470,8.900000e+07,48424.0,NaN,NaN,NaN,UM,..,687.500000
4,AGO,Angola,49.1,8.10,77.7,45.290001,7.333445,65.269997,16220.0,16140.0,...,6.059469,0.571521,4.713840e+09,31273533.0,49.1,9.522,21.620001,LM,2.95,137.850460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,VIR,Virgin Islands (U.S.),NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,NaN,...,NaN,NaN,NaN,107001.0,NaN,12.245,93.419998,H,..,688.524540
196,PSE,West Bank and Gaza,NaN,NaN,NaN,100.000000,100.000000,100.000000,NaN,NaN,...,11.547252,0.341530,1.879500e+09,4569087.0,15.3,26.260,69.739998,LM,2.55,524.999940
197,YEM,"Yemen, Rep.",61.5,42.90,93.7,62.000000,NaN,NaN,4360.0,2980.0,...,NaN,NaN,NaN,30790513.0,26.6,13.076,45.299999,L,..,542.424260
198,ZMB,Zambia,11.9,2.10,24.6,40.317890,12.466436,76.461876,10330.0,13510.0,...,6.847766,4.071350,1.801756e+09,17835893.0,15.6,5.030,24.860001,LM,7.35,130.192200


In [155]:
variables = [
    'Country Code', 
    'Country Name',

#pollution variables
    'Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS]',
    'Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]',
    'Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]',

    'Access to electricity (% of population) [EG.ELC.ACCS.ZS]',
    'Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]',
    'Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]',

    'Renewable energy consumption (% of total final energy consumption) [EG.FEC.RNEW.ZS]',

    'Total greenhouse gas emissions (kt of CO2 equivalent) [EN.ATM.GHGT.KT.CE]',

    'CO2 emissions (kg per 2017 PPP $ of GDP) [EN.ATM.CO2E.PP.GD.KD]',

    'Agricultural methane emissions (thousand metric tons of CO2 equivalent) [EN.ATM.METH.AG.KT.CE]',
    'Agricultural nitrous oxide emissions (thousand metric tons of CO2 equivalent) [EN.ATM.NOXE.AG.KT.CE]',

    'Methane emissions (kt of CO2 equivalent) [EN.ATM.METH.KT.CE]',
    'Nitrous oxide emissions (thousand metric tons of CO2 equivalent) [EN.ATM.NOXE.KT.CE]',

    'Energy intensity level of primary energy (MJ/$2017 PPP GDP) [EG.EGY.PRIM.PP.KD]',
    'Carbon intensity of electricity (gCO2/kWh)',


#economics variables
    'Age dependency ratio, old (% of working-age population) [SP.POP.DPND.OL]',
    'Employers, total (% of total employment) (modeled ILO estimate) [SL.EMP.MPYR.ZS]',
    'Exports of goods and services (current US$) [NE.EXP.GNFS.CD]',
    'Imports of goods and services (current US$) [NE.IMP.GNFS.CD]',
    'Final consumption expenditure (current US$) [NE.CON.TOTL.CD]',
    'GDP, PPP (constant 2017 international $) [NY.GDP.MKTP.PP.KD]',
    'GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]',
    'GNI, PPP (current international $) [NY.GNP.MKTP.PP.CD]',
    'Industry (including construction), value added (% of GDP) [NV.IND.TOTL.ZS]',
    'Population, total [SP.POP.TOTL]',
    'Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]',

    'Income Group'

]

In [166]:
num_variables = [

#pollution variables
    'Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS]',
    'Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]',
    'Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]',

    'Access to electricity (% of population) [EG.ELC.ACCS.ZS]',
    'Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]',
    'Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]',

    'Renewable energy consumption (% of total final energy consumption) [EG.FEC.RNEW.ZS]',

    'Total greenhouse gas emissions (kt of CO2 equivalent) [EN.ATM.GHGT.KT.CE]',

    'CO2 emissions (kg per 2017 PPP $ of GDP) [EN.ATM.CO2E.PP.GD.KD]',

    'Agricultural methane emissions (thousand metric tons of CO2 equivalent) [EN.ATM.METH.AG.KT.CE]',
    'Agricultural nitrous oxide emissions (thousand metric tons of CO2 equivalent) [EN.ATM.NOXE.AG.KT.CE]',

    'Methane emissions (kt of CO2 equivalent) [EN.ATM.METH.KT.CE]',
    'Nitrous oxide emissions (thousand metric tons of CO2 equivalent) [EN.ATM.NOXE.KT.CE]',

    'Energy intensity level of primary energy (MJ/$2017 PPP GDP) [EG.EGY.PRIM.PP.KD]',
    'Carbon intensity of electricity (gCO2/kWh)',


#economics variables
    'Age dependency ratio, old (% of working-age population) [SP.POP.DPND.OL]',
    'Employers, total (% of total employment) (modeled ILO estimate) [SL.EMP.MPYR.ZS]',
    'Exports of goods and services (current US$) [NE.EXP.GNFS.CD]',
    'Imports of goods and services (current US$) [NE.IMP.GNFS.CD]',
    'Final consumption expenditure (current US$) [NE.CON.TOTL.CD]',
    'GDP, PPP (constant 2017 international $) [NY.GDP.MKTP.PP.KD]',
    'GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]',
    'GNI, PPP (current international $) [NY.GNP.MKTP.PP.CD]',
    'Industry (including construction), value added (% of GDP) [NV.IND.TOTL.ZS]',
    'Population, total [SP.POP.TOTL]',
    'Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]'

]

In [156]:
carbon_intensity_added[variables]

,Country Code,Country Name,Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS],"Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]","Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]",Access to electricity (% of population) [EG.ELC.ACCS.ZS],"Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]","Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]",Renewable energy consumption (% of total final energy consumption) [EG.FEC.RNEW.ZS],Total greenhouse gas emissions (kt of CO2 equivalent) [EN.ATM.GHGT.KT.CE],...,Exports of goods and services (current US$) [NE.EXP.GNFS.CD],Imports of goods and services (current US$) [NE.IMP.GNFS.CD],Final consumption expenditure (current US$) [NE.CON.TOTL.CD],"GDP, PPP (constant 2017 international $) [NY.GDP.MKTP.PP.KD]",GDP growth (annual %) [NY.GDP.MKTP.KD.ZG],"GNI, PPP (current international $) [NY.GNP.MKTP.PP.CD]","Industry (including construction), value added (% of GDP) [NV.IND.TOTL.ZS]","Population, total [SP.POP.TOTL]","Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]",Income Group
0,AFG,Afghanistan,30.9,13.85,81.6,96.616135,95.586174,99.626022,17.96,27690.000534,...,NaN,NaN,NaN,7.560042e+10,1.189228,7.821093e+10,13.387247,36686784.0,17.660000,L
1,ALB,Albania,79.8,62.50,92.2,100.000000,100.000000,100.000000,38.11,9449.999809,...,4.785178e+09,6.856193e+09,1.370054e+10,3.817186e+10,4.019360,3.864383e+10,21.279306,2866376.0,44.520000,UM
2,DZA,Algeria,99.6,98.70,100.0,99.697838,99.071304,99.933952,0.19,278029.998779,...,4.523397e+10,5.632890e+10,1.036736e+11,4.916310e+11,1.200000,4.907023e+11,38.805186,41927007.0,68.019997,UM
3,ASM,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,0.49,NaN,...,4.360000e+08,6.620000e+08,NaN,NaN,2.671119,NaN,NaN,48424.0,NaN,UM
4,AGO,Angola,49.1,8.10,77.7,45.290001,7.333445,65.269997,57.36,75239.997864,...,3.176775e+10,1.986966e+10,5.199370e+10,2.151179e+11,-1.316312,2.032423e+11,47.931542,31273533.0,21.620001,LM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,VIR,Virgin Islands (U.S.),NaN,NaN,NaN,100.000000,100.000000,100.000000,3.43,NaN,...,2.084000e+09,4.163000e+09,4.352000e+09,NaN,1.904762,NaN,NaN,107001.0,93.419998,H
196,PSE,West Bank and Gaza,NaN,NaN,NaN,100.000000,100.000000,100.000000,12.73,NaN,...,2.598000e+09,9.023700e+09,1.804750e+10,2.886845e+10,1.227077,3.461983e+10,19.284126,4569087.0,69.739998,LM
197,YEM,"Yemen, Rep.",61.5,42.90,93.7,62.000000,NaN,NaN,3.60,23860.000610,...,1.308592e+09,9.344000e+09,NaN,NaN,0.752448,NaN,15.209188,30790513.0,45.299999,L
198,ZMB,Zambia,11.9,2.10,24.6,40.317890,12.466436,76.461876,82.28,37560.001373,...,9.987197e+09,9.717124e+09,1.517567e+10,6.110486e+10,4.034494,6.159734e+10,34.485322,17835893.0,24.860001,LM


In [157]:
data_reduced = carbon_intensity_added[variables]

In [173]:
data_reduced = data_reduced.replace(to_replace='..', value=np.NaN)

In [176]:
data_reduced_numerical = data_reduced[num_variables]

In [177]:
for e in data_reduced_numerical['Energy intensity level of primary energy (MJ/$2017 PPP GDP) [EG.EGY.PRIM.PP.KD]']:
    print(e)

2.44
2.57
5.19
nan
2.95
3.57
3.31
3.51
3.07
4.39
2.81
4.32
2.72
8.27
2.3
3.87
6.29
3.82
5.69
6.05
8.22
3.99
6.54
2.76
3.93
nan
5.85
4.98
4.6
7.71
2.68
4.61
4.43
7.1
1.9
8.56
3.85
3.49
6.39
2.29
3.12
13.82
6.12
1.98
3.48
3.13
1.37
2.74
4.28
2.14
2.02
2.94
2.06
3.1
3.59
3.27
3.72
nan
5.54
4.5
7.88
nan
2.12
5.37
3.4
nan
6.64
3.08
3.82
2.87
2.92
3.0
nan
2.54
nan
4.25
5.51
8.93
3.86
5.78
4.58
1.31
3.66
12.78
4.4
3.19
10.39
5.67
1.41
2.65
2.48
4.28
3.45
3.82
6.65
5.26
6.03
nan
5.46
7.07
5.88
4.7
3.35
3.36
8.08
13.59
7.16
3.22
2.37
0.47
8.5
3.28
4.5
2.56
4.76
1.36
3.35
2.15
3.09
5.09
6.37
3.42
3.18
12.07
3.52
3.44
5.15
5.68
3.17
nan
3.93
4.46
5.03
6.16
3.23
3.66
7.79
4.48
1.39
5.31
3.4
2.56
2.81
3.68
2.63
0.55
6.46
2.52
8.25
4.01
4.23
3.49
5.58
3.48
5.28
3.16
5.51
2.51
4.28
3.67
4.28
11.69
7.73
nan
2.77
1.76
2.59
2.86
2.67
4.47
3.57
3.97
1.66
11.72
4.73
6.49
4.5
2.12
8.48
2.87
19.34
3.85
2.59
13.33
4.92
10.17
7.51
4.11
2.41
4.61
2.79
9.05
3.8
nan
4.74
nan
2.55
nan
7.35
9.83


In [179]:
#MISSING VALUES IMPUTATION
# Introduce IterativeImputer with an estimator
imp = IterativeImputer(estimator=RandomForestRegressor())

In [180]:
# Fit to the dataset containing missing values
imp.fit(data_reduced_numerical)

C:\Users\isabe\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


IterativeImputer(estimator=RandomForestRegressor())

In [181]:
# Transform the dataset containing missing values
filled = pd.DataFrame(imp.transform(data_reduced_numerical), columns = data_reduced_numerical.columns)

In [182]:
filled

,Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS],"Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]","Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]",Access to electricity (% of population) [EG.ELC.ACCS.ZS],"Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]","Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]",Renewable energy consumption (% of total final energy consumption) [EG.FEC.RNEW.ZS],Total greenhouse gas emissions (kt of CO2 equivalent) [EN.ATM.GHGT.KT.CE],CO2 emissions (kg per 2017 PPP $ of GDP) [EN.ATM.CO2E.PP.GD.KD],Agricultural methane emissions (thousand metric tons of CO2 equivalent) [EN.ATM.METH.AG.KT.CE],...,"Employers, total (% of total employment) (modeled ILO estimate) [SL.EMP.MPYR.ZS]",Exports of goods and services (current US$) [NE.EXP.GNFS.CD],Imports of goods and services (current US$) [NE.IMP.GNFS.CD],Final consumption expenditure (current US$) [NE.CON.TOTL.CD],"GDP, PPP (constant 2017 international $) [NY.GDP.MKTP.PP.KD]",GDP growth (annual %) [NY.GDP.MKTP.KD.ZG],"GNI, PPP (current international $) [NY.GNP.MKTP.PP.CD]","Industry (including construction), value added (% of GDP) [NV.IND.TOTL.ZS]","Population, total [SP.POP.TOTL]","Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]"
0,30.900,13.8500,81.600,96.616135,95.586174,99.626022,17.96,27690.000534,0.080291,10540.0,...,2.7200,1.062330e+11,1.119149e+11,1.921012e+11,7.560042e+10,1.189228,7.821093e+10,13.387247,36686784.0,17.660000
1,79.800,62.5000,92.200,100.000000,100.000000,100.000000,38.11,9449.999809,0.133868,2010.0,...,3.2900,4.785178e+09,6.856193e+09,1.370054e+10,3.817186e+10,4.019360,3.864383e+10,21.279306,2866376.0,44.520000
2,99.600,98.7000,100.000,99.697838,99.071304,99.933952,0.19,278029.998779,0.336716,6820.0,...,4.6300,4.523397e+10,5.632890e+10,1.036736e+11,4.916310e+11,1.200000,4.907023e+11,38.805186,41927007.0,68.019997
3,81.168,59.2630,93.575,84.048066,73.536247,94.733865,0.49,243760.104065,0.550819,23820.4,...,2.5561,4.360000e+08,6.620000e+08,1.677895e+10,9.545896e+11,2.671119,9.707024e+11,25.338415,48424.0,58.927900
4,49.100,8.1000,77.700,45.290001,7.333445,65.269997,57.36,75239.997864,0.111381,16220.0,...,5.2600,3.176775e+10,1.986966e+10,5.199370e+10,2.151179e+11,-1.316312,2.032423e+11,47.931542,31273533.0,21.620001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,55.394,32.2690,78.416,100.000000,100.000000,100.000000,3.43,226545.302620,0.161743,33739.7,...,3.3600,2.084000e+09,4.163000e+09,4.352000e+09,9.472831e+11,1.904762,9.697226e+11,13.940961,107001.0,93.419998
196,45.157,18.7335,70.111,100.000000,100.000000,100.000000,12.73,40982.100382,0.177872,23361.5,...,7.0800,2.598000e+09,9.023700e+09,1.804750e+10,2.886845e+10,1.227077,3.461983e+10,19.284126,4569087.0,69.739998
197,61.500,42.9000,93.700,62.000000,37.545254,87.139853,3.60,23860.000610,0.248813,4360.0,...,7.0700,1.308592e+09,9.344000e+09,3.643334e+10,8.711139e+11,0.752448,9.142263e+11,15.209188,30790513.0,45.299999
198,11.900,2.1000,24.600,40.317890,12.466436,76.461876,82.28,37560.001373,0.126504,10330.0,...,0.2500,9.987197e+09,9.717124e+09,1.517567e+10,6.110486e+10,4.034494,6.159734e+10,34.485322,17835893.0,24.860001


In [184]:
data_reduced.loc[:, num_variables] = filled

In [187]:
data_reduced

,Country Code,Country Name,Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS],"Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]","Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]",Access to electricity (% of population) [EG.ELC.ACCS.ZS],"Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]","Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]",Renewable energy consumption (% of total final energy consumption) [EG.FEC.RNEW.ZS],Total greenhouse gas emissions (kt of CO2 equivalent) [EN.ATM.GHGT.KT.CE],...,Exports of goods and services (current US$) [NE.EXP.GNFS.CD],Imports of goods and services (current US$) [NE.IMP.GNFS.CD],Final consumption expenditure (current US$) [NE.CON.TOTL.CD],"GDP, PPP (constant 2017 international $) [NY.GDP.MKTP.PP.KD]",GDP growth (annual %) [NY.GDP.MKTP.KD.ZG],"GNI, PPP (current international $) [NY.GNP.MKTP.PP.CD]","Industry (including construction), value added (% of GDP) [NV.IND.TOTL.ZS]","Population, total [SP.POP.TOTL]","Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]",Income Group
0,AFG,Afghanistan,30.900,13.8500,81.600,96.616135,95.586174,99.626022,17.96,27690.000534,...,1.062330e+11,1.119149e+11,1.921012e+11,7.560042e+10,1.189228,7.821093e+10,13.387247,36686784.0,17.660000,L
1,ALB,Albania,79.800,62.5000,92.200,100.000000,100.000000,100.000000,38.11,9449.999809,...,4.785178e+09,6.856193e+09,1.370054e+10,3.817186e+10,4.019360,3.864383e+10,21.279306,2866376.0,44.520000,UM
2,DZA,Algeria,99.600,98.7000,100.000,99.697838,99.071304,99.933952,0.19,278029.998779,...,4.523397e+10,5.632890e+10,1.036736e+11,4.916310e+11,1.200000,4.907023e+11,38.805186,41927007.0,68.019997,UM
3,ASM,American Samoa,81.168,59.2630,93.575,84.048066,73.536247,94.733865,0.49,243760.104065,...,4.360000e+08,6.620000e+08,1.677895e+10,9.545896e+11,2.671119,9.707024e+11,25.338415,48424.0,58.927900,UM
4,AGO,Angola,49.100,8.1000,77.700,45.290001,7.333445,65.269997,57.36,75239.997864,...,3.176775e+10,1.986966e+10,5.199370e+10,2.151179e+11,-1.316312,2.032423e+11,47.931542,31273533.0,21.620001,LM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,VIR,Virgin Islands (U.S.),55.394,32.2690,78.416,100.000000,100.000000,100.000000,3.43,226545.302620,...,2.084000e+09,4.163000e+09,4.352000e+09,9.472831e+11,1.904762,9.697226e+11,13.940961,107001.0,93.419998,H
196,PSE,West Bank and Gaza,45.157,18.7335,70.111,100.000000,100.000000,100.000000,12.73,40982.100382,...,2.598000e+09,9.023700e+09,1.804750e+10,2.886845e+10,1.227077,3.461983e+10,19.284126,4569087.0,69.739998,LM
197,YEM,"Yemen, Rep.",61.500,42.9000,93.700,62.000000,37.545254,87.139853,3.60,23860.000610,...,1.308592e+09,9.344000e+09,3.643334e+10,8.711139e+11,0.752448,9.142263e+11,15.209188,30790513.0,45.299999,L
198,ZMB,Zambia,11.900,2.1000,24.600,40.317890,12.466436,76.461876,82.28,37560.001373,...,9.987197e+09,9.717124e+09,1.517567e+10,6.110486e+10,4.034494,6.159734e+10,34.485322,17835893.0,24.860001,LM


In [196]:
data_reduced.to_excel(r'C:\Users\isabe\OneDrive - Università degli Studi di Milano\second year\1st trimester\adv multivariate statistics\AMS\kaya identity analysis\data_reduce_filled.xlsx', index = False)

In [197]:
dudu = pd.read_excel("data_reduce_filled.xlsx")

In [198]:
dudu

,Country Code,Country Name,Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS],"Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]","Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.ACCS.UR.ZS]",Access to electricity (% of population) [EG.ELC.ACCS.ZS],"Access to electricity, rural (% of rural population) [EG.ELC.ACCS.RU.ZS]","Access to electricity, urban (% of urban population) [EG.ELC.ACCS.UR.ZS]",Renewable energy consumption (% of total final energy consumption) [EG.FEC.RNEW.ZS],Total greenhouse gas emissions (kt of CO2 equivalent) [EN.ATM.GHGT.KT.CE],...,Exports of goods and services (current US$) [NE.EXP.GNFS.CD],Imports of goods and services (current US$) [NE.IMP.GNFS.CD],Final consumption expenditure (current US$) [NE.CON.TOTL.CD],"GDP, PPP (constant 2017 international $) [NY.GDP.MKTP.PP.KD]",GDP growth (annual %) [NY.GDP.MKTP.KD.ZG],"GNI, PPP (current international $) [NY.GNP.MKTP.PP.CD]","Industry (including construction), value added (% of GDP) [NV.IND.TOTL.ZS]","Population, total [SP.POP.TOTL]","Wage and salaried workers, total (% of total employment) (modeled ILO estimate) [SL.EMP.WORK.ZS]",Income Group
0,AFG,Afghanistan,30.900,13.8500,81.600,96.616135,95.586174,99.626022,17.96,27690.000534,...,1.062330e+11,1.119149e+11,1.921012e+11,7.560042e+10,1.189228,7.821093e+10,13.387247,36686784,17.660000,L
1,ALB,Albania,79.800,62.5000,92.200,100.000000,100.000000,100.000000,38.11,9449.999809,...,4.785178e+09,6.856193e+09,1.370054e+10,3.817186e+10,4.019360,3.864383e+10,21.279306,2866376,44.520000,UM
2,DZA,Algeria,99.600,98.7000,100.000,99.697838,99.071304,99.933952,0.19,278029.998779,...,4.523397e+10,5.632890e+10,1.036736e+11,4.916310e+11,1.200000,4.907023e+11,38.805186,41927007,68.019997,UM
3,ASM,American Samoa,81.168,59.2630,93.575,84.048066,73.536247,94.733865,0.49,243760.104065,...,4.360000e+08,6.620000e+08,1.677895e+10,9.545896e+11,2.671119,9.707024e+11,25.338415,48424,58.927900,UM
4,AGO,Angola,49.100,8.1000,77.700,45.290001,7.333445,65.269997,57.36,75239.997864,...,3.176775e+10,1.986966e+10,5.199370e+10,2.151179e+11,-1.316312,2.032423e+11,47.931542,31273533,21.620001,LM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,VIR,Virgin Islands (U.S.),55.394,32.2690,78.416,100.000000,100.000000,100.000000,3.43,226545.302620,...,2.084000e+09,4.163000e+09,4.352000e+09,9.472831e+11,1.904762,9.697226e+11,13.940961,107001,93.419998,H
196,PSE,West Bank and Gaza,45.157,18.7335,70.111,100.000000,100.000000,100.000000,12.73,40982.100382,...,2.598000e+09,9.023700e+09,1.804750e+10,2.886845e+10,1.227077,3.461983e+10,19.284126,4569087,69.739998,LM
197,YEM,"Yemen, Rep.",61.500,42.9000,93.700,62.000000,37.545254,87.139853,3.60,23860.000610,...,1.308592e+09,9.344000e+09,3.643334e+10,8.711139e+11,0.752448,9.142263e+11,15.209188,30790513,45.299999,L
198,ZMB,Zambia,11.900,2.1000,24.600,40.317890,12.466436,76.461876,82.28,37560.001373,...,9.987197e+09,9.717124e+09,1.517567e+10,6.110486e+10,4.034494,6.159734e+10,34.485322,17835893,24.860001,LM


In [199]:
data_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 29 columns):
 #   Column                                                                                                 Non-Null Count  Dtype   
---  ------                                                                                                 --------------  -----   
 0   Country Code                                                                                           200 non-null    object  
 1   Country Name                                                                                           200 non-null    object  
 2   Access to clean fuels and technologies for cooking (% of population) [EG.CFT.ACCS.ZS]                  200 non-null    float64 
 3   Access to clean fuels and technologies for cooking, rural (% of rural population) [EG.CFT.ACCS.RU.ZS]  200 non-null    float64 
 4   Access to clean fuels and technologies for cooking, urban (% of urban population) [EG.CFT.AC